In [1]:
from menelaus.drift_detector import BatchDetector, Ensembler
from typing import List
from toolz import pipe

In [3]:
class BatchEnsembler(BatchDetector, Ensembler):
    def __init__(self, detectors: dict, pre_processors: List, post_processors: List):
        BatchDetector.__init__(self)
        Ensembler.__init__(self, detectors, pre_processors, post_processors)

    def update(self, X, y_true, y_pred):
        self = pipe(self, *self.pre_processors)     # how does it use extra args? read docs

        for det_key in self.detectors:
            det = self.detectors[det_key]
            det.update(X, y_true, y_pred)
            # post processing to collect, for ... post_processors <- this is weird

        self = pipe(self, *self.post_processors)    # how can we move common logic into Ensembler
        return self                                 #   while knowing exactly what happens when
                                                    #   the parent update called from here says
                                                    #   self = ...

    def reset(self):
        BatchDetector.reset(self)

    def set_reference(self):
        # really not sure what to do here
        pass


# I think "operator" functions should go in class files in detector code
be = BatchEnsembler({}, [], [])
be = be.update(0,0,0)